In [2]:
#Создаем соединение
import psycopg2
import getpass

conn=psycopg2.connect(user='postgres',
                     database='postgres',
                     host='localhost',
                     port='5432',
                     password=getpass.getpass('Insert password'))

Insert password········


In [3]:
# Получение данных из БД PostgreSQL, с которыми будем дальше работать, при помощи запроса SQL. 
# Запрашиваем данные из таблиц bond_description и quotes, объединенных по общему полю "ISIN".
# Нам необходимо все, что потребуется для создания матрицы платежей: размер купона, даты выплат купона, номинал, дата погашения и проч (см.результат ниже)
# При этом нужно исключить облигации, которые погашены, размещаются в данный момент и по которым случился дефолт
import pandas as pd
query_bonds="SELECT DISTINCT bond_description.\"ISINCODE\",bond_description.\"ISSUERNAME\", bond_description.\"HAVEOFFER\", bond_description.\"COUPONTYPE\", bond_description.\"SECURITYTYPE\", quotes.\"BUYBACKDATE\", quotes.\"CPN\", quotes.\"CPN_DATE\", quotes.\"CPN_PERIOD\", quotes.\"FACEVALUE\", quotes.\"MAT_DATE\" FROM bond_description INNER JOIN quotes ON bond_description.\"ISINCODE\"=quotes.\"ISIN\" WHERE \"STATUS\"!='Погашен' and \"STATUS\"!='Размещается' and \"HAVEDEFAULT\"='0'"
bonds_desc=pd.read_sql_query(query_bonds,conn)
bonds_desc.shape

(4468, 11)

In [51]:
# Выводим первые 10 строк для проверки
bonds_desc.head(10)

,ISINCODE,ISSUERNAME,HAVEOFFER,COUPONTYPE,SECURITYTYPE,BUYBACKDATE,CPN,CPN_DATE,CPN_PERIOD,FACEVALUE,MAT_DATE
0,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,4.99,2012-08-23,182.0,1000.0,2016-02-18
1,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2011-02-24,182.0,1000.0,2016-02-18
2,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2011-08-25,182.0,1000.0,2016-02-18
3,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2012-02-23,182.0,1000.0,2016-02-18
4,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2012-08-23,182.0,1000.0,2016-02-18
5,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2009-08-27,NaN,1000.0,2016-02-18
6,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2010-02-25,NaN,1000.0,2016-02-18
7,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2010-08-26,NaN,1000.0,2016-02-18
8,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2011-02-24,NaN,1000.0,2016-02-18
9,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,None,NaN,1000.0,2016-02-18


In [6]:
# Исходные данные для облигаций с фикс или пост купоном
fixed_coup=bonds_desc[['ISINCODE','CPN','CPN_DATE','MAT_DATE','FACEVALUE']][(bonds_desc['COUPONTYPE']=='Постоянный')|(bonds_desc['COUPONTYPE']=='Фиксированный')]
fixed_coup['CPN'].fillna(0, inplace=True)
fixed_coup.head(10)

,ISINCODE,CPN,CPN_DATE,MAT_DATE,FACEVALUE
77,RU000A0JNYN1,34.9,2013-06-11,2022-06-11,1000.0
78,RU000A0JNYN1,34.9,2013-12-11,2022-06-11,1000.0
79,RU000A0JNYN1,34.9,2014-06-11,2022-06-11,1000.0
80,RU000A0JNYN1,34.9,2014-12-11,2022-06-11,1000.0
81,RU000A0JNYN1,34.9,2015-06-11,2022-06-11,1000.0
82,RU000A0JNYN1,34.9,2015-12-11,2022-06-11,1000.0
83,RU000A0JNYN1,35.1,2012-12-11,2022-06-11,1000.0
84,RU000A0JNYN1,35.1,2013-06-11,2022-06-11,1000.0
85,RU000A0JNYN1,35.1,2013-12-11,2022-06-11,1000.0
86,RU000A0JNYN1,35.1,2014-06-11,2022-06-11,1000.0


In [12]:
# Считаем денежный поток по облигациям с фикс или пост купоном
import numpy as np
fixed_coup['CF']=np.where(fixed_coup['CPN_DATE']==fixed_coup['MAT_DATE'], fixed_coup['CPN']+fixed_coup['FACEVALUE'], fixed_coup['CPN'])
fixed_coup.head(10)

,ISINCODE,CPN,CPN_DATE,MAT_DATE,FACEVALUE,CF
77,RU000A0JNYN1,34.9,2013-06-11,2022-06-11,1000.0,34.9
78,RU000A0JNYN1,34.9,2013-12-11,2022-06-11,1000.0,34.9
79,RU000A0JNYN1,34.9,2014-06-11,2022-06-11,1000.0,34.9
80,RU000A0JNYN1,34.9,2014-12-11,2022-06-11,1000.0,34.9
81,RU000A0JNYN1,34.9,2015-06-11,2022-06-11,1000.0,34.9
82,RU000A0JNYN1,34.9,2015-12-11,2022-06-11,1000.0,34.9
83,RU000A0JNYN1,35.1,2012-12-11,2022-06-11,1000.0,35.1
84,RU000A0JNYN1,35.1,2013-06-11,2022-06-11,1000.0,35.1
85,RU000A0JNYN1,35.1,2013-12-11,2022-06-11,1000.0,35.1
86,RU000A0JNYN1,35.1,2014-06-11,2022-06-11,1000.0,35.1


In [9]:
# Для проверки (Код ниже проверяет, правильно ли заполнились ячейки по условию выше )

# fixed_coup.loc[fixed_coup['CF'] != (fixed_coup['CPN']+fixed_coup['FACEVALUE'])].head(10)

,ISINCODE,CPN,CPN_DATE,MAT_DATE,FACEVALUE,CF
77,RU000A0JNYN1,34.9,2013-06-11,2022-06-11,1000.0,34.9
78,RU000A0JNYN1,34.9,2013-12-11,2022-06-11,1000.0,34.9
79,RU000A0JNYN1,34.9,2014-06-11,2022-06-11,1000.0,34.9
80,RU000A0JNYN1,34.9,2014-12-11,2022-06-11,1000.0,34.9
81,RU000A0JNYN1,34.9,2015-06-11,2022-06-11,1000.0,34.9
82,RU000A0JNYN1,34.9,2015-12-11,2022-06-11,1000.0,34.9
83,RU000A0JNYN1,35.1,2012-12-11,2022-06-11,1000.0,35.1
84,RU000A0JNYN1,35.1,2013-06-11,2022-06-11,1000.0,35.1
85,RU000A0JNYN1,35.1,2013-12-11,2022-06-11,1000.0,35.1
86,RU000A0JNYN1,35.1,2014-06-11,2022-06-11,1000.0,35.1


In [11]:
#Для проверки (вторая часть условия)
# fixed_coup.loc[fixed_coup['CF'] != fixed_coup['CPN']].head(10)

,ISINCODE,CPN,CPN_DATE,MAT_DATE,FACEVALUE,CF
958,RU000A0JR530,21.0800,2015-11-25,2015-11-25,500.0,521.0800
959,RU000A0JR530,21.4200,2015-11-25,2015-11-25,500.0,521.4200
960,RU000A0JR530,21.4247,2015-11-25,2015-11-25,500.0,521.4247
981,RU000A0JR6K5,9.2700,2015-12-17,2015-12-17,400.0,409.2700
982,RU000A0JR6K5,9.2745,2015-12-17,2015-12-17,400.0,409.2745
1013,RU000A0JR852,42.3800,2016-02-02,2016-02-02,1000.0,1042.3800
1014,RU000A0JR852,42.3836,2016-02-02,2016-02-02,1000.0,1042.3836
1383,RU000A0JRW93,43.6400,2016-01-31,2016-01-31,1000.0,1043.6400
1388,RU000A0JRW93,44.3600,2016-01-31,2016-01-31,1000.0,1044.3600
1389,RU000A0JRW93,44.3616,2016-01-31,2016-01-31,1000.0,1044.3616


In [50]:
# Список аргументов:
# код эмитента - isin
# дата матрицы платежей - cpndate
# Это попытка написания функции для облигаций с пост или фикс купоном (хотела написать сначала функцию именно на этом виде купона,т.к. наиболее понятен)
def my_fun(isin, cpndate):
    fcoup_formatrix=fcoup_formatrix.pivot_table(index=['ISINCODE'],
                      columns='CPN_DATE',
                     values='CF').reset_index().rename_axis(None, axis=1)
    fcoup_formatrix=fcoup_formatrix.fillna(0)

    results={'ISINCODE': isin, 'CPN_DATE': cpndate, 'FMATRIX': fcoup_formatrix }
    return results

In [49]:
# Но При вызове функции выдается ошибка 
x=my_fun('\'RU000A0JR6K5\'', '\'2016-01-31\'')
x['FMATRIX']


UnboundLocalError: local variable 'fcoup_formatrix1' referenced before assignment

In [54]:
# Здесь и ниже я привожу пример того, что пыталась получить от функции, как бы это примерно выглядело, но через сводную таблицу
fcoup_formatrix=fixed_coup[['ISINCODE','CPN_DATE','CF']]
fcoup_formatrix.head(10)

,ISINCODE,CPN_DATE,CF
77,RU000A0JNYN1,2013-06-11,34.9
78,RU000A0JNYN1,2013-12-11,34.9
79,RU000A0JNYN1,2014-06-11,34.9
80,RU000A0JNYN1,2014-12-11,34.9
81,RU000A0JNYN1,2015-06-11,34.9
82,RU000A0JNYN1,2015-12-11,34.9
83,RU000A0JNYN1,2012-12-11,35.1
84,RU000A0JNYN1,2013-06-11,35.1
85,RU000A0JNYN1,2013-12-11,35.1
86,RU000A0JNYN1,2014-06-11,35.1


In [55]:
fcoup_formatrix=fcoup_formatrix.pivot_table(index=['ISINCODE'],
                      columns='CPN_DATE',
                     values='CF').reset_index().rename_axis(None, axis=1).head(10)
fcoup_formatrix=fcoup_formatrix.fillna(0)
fcoup_formatrix

,ISINCODE,2009-06-11,2009-06-14,2009-09-22,2009-12-11,2009-12-14,2009-12-28,2010-03-22,2010-06-11,2010-06-14,...,2016-03-25,2016-03-29,2016-04-04,2016-04-06,2016-04-07,2016-04-08,2016-04-14,2016-04-19,2016-04-22,2016-04-27
0,RU000A0JNYN1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,RU000A0JNYP6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,RU000A0JPZ50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,RU000A0JQ6S9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,RU000A0JR4H6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,RU000A0JR530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,RU000A0JR6K5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,RU000A0JR852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,RU000A0JRGR0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,RU000A0JRJN3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
from IPython.display import display
pd.options.display.max_columns = None
display(fcoup_formatrix)

,ISINCODE,2009-06-11,2009-06-14,2009-09-22,2009-12-11,2009-12-14,2009-12-28,2010-03-22,2010-06-11,2010-06-14,2010-06-28,2010-09-22,2010-12-11,2010-12-14,2010-12-27,2011-03-22,2011-03-24,2011-05-17,2011-05-25,2011-06-11,2011-06-14,2011-06-23,2011-06-27,2011-06-29,2011-08-09,2011-09-22,2011-11-09,2011-11-15,2011-11-25,2011-12-11,2011-12-14,2011-12-22,2011-12-26,2011-12-28,2012-02-07,2012-03-14,2012-03-21,2012-03-22,2012-05-09,2012-05-15,2012-05-25,2012-05-31,2012-06-11,2012-06-14,2012-06-20,2012-06-21,2012-06-25,2012-06-27,2012-08-07,2012-09-05,2012-09-12,2012-09-19,2012-09-20,2012-09-22,2012-10-19,2012-11-07,2012-11-13,2012-11-25,2012-11-28,2012-11-29,2012-12-04,2012-12-11,2012-12-19,2012-12-20,2012-12-24,2012-12-26,2013-01-04,2013-02-05,2013-02-07,2013-02-27,2013-03-12,2013-03-13,2013-03-19,2013-03-20,2013-03-21,2013-03-22,2013-04-20,2013-04-23,2013-05-02,2013-05-08,2013-05-14,2013-05-21,2013-05-25,2013-05-29,2013-05-30,2013-05-31,2013-06-04,2013-06-11,2013-06-12,2013-06-18,2013-06-19,2013-06-20,2013-06-24,2013-06-26,2013-07-05,2013-07-23,2013-07-30,2013-07-31,2013-08-01,2013-08-02,2013-08-06,2013-08-08,2013-08-13,2013-08-14,2013-08-15,2013-08-20,2013-08-27,2013-08-28,2013-09-01,2013-09-05,2013-09-10,2013-09-11,2013-09-17,2013-09-18,2013-09-19,2013-09-22,2013-10-01,2013-10-04,2013-10-20,2013-10-22,2013-10-31,2013-11-06,2013-11-12,2013-11-19,2013-11-25,2013-11-27,2013-11-28,2013-11-29,2013-12-03,2013-12-04,2013-12-09,2013-12-10,2013-12-11,2013-12-17,2013-12-18,2013-12-19,2013-12-23,2013-12-25,2014-01-02,2014-01-03,2014-01-20,2014-01-21,2014-01-23,2014-01-27,2014-01-29,2014-01-30,2014-01-31,2014-02-04,2014-02-06,2014-02-11,2014-02-12,2014-02-13,2014-02-18,2014-02-20,2014-02-25,2014-02-26,2014-02-27,2014-03-01,2014-03-04,2014-03-06,2014-03-11,2014-03-12,2014-03-18,2014-03-19,2014-03-20,2014-03-22,2014-03-27,2014-03-28,2014-04-01,2014-04-03,2014-04-04,2014-04-10,2014-04-21,2014-04-22,2014-04-24,2014-04-28,2014-04-30,2014-05-01,2014-05-07,2014-05-13,2014-05-20,2014-05-21,2014-05-22,2014-05-25,2014-05-26,2014-05-27,2014-05-28,2014-05-29,2014-05-30,2014-06-03,2014-06-04,2014-06-09,2014-06-10,2014-06-11,2014-06-17,2014-06-18,2014-06-19,2014-06-23,2014-06-25,2014-06-26,2014-07-03,2014-07-04,2014-07-21,2014-07-22,2014-07-24,2014-07-28,2014-07-30,2014-07-31,2014-08-01,2014-08-05,2014-08-07,2014-08-12,2014-08-13,2014-08-14,2014-08-19,2014-08-21,2014-08-26,2014-08-27,2014-08-28,2014-09-01,2014-09-02,2014-09-04,2014-09-09,2014-09-10,2014-09-16,2014-09-17,2014-09-18,2014-09-22,2014-09-25,2014-09-26,2014-09-30,2014-10-02,2014-10-04,2014-10-09,2014-10-16,2014-10-20,2014-10-21,2014-10-24,2014-10-27,2014-10-29,2014-10-30,2014-11-05,2014-11-11,2014-11-18,2014-11-19,2014-11-20,2014-11-25,2014-11-26,2014-11-27,2014-11-28,2014-12-02,2014-12-03,2014-12-04,2014-12-08,2014-12-09,2014-12-10,2014-12-11,2014-12-16,2014-12-17,2014-12-18,2014-12-22,2014-12-24,2014-12-25,2015-01-01,2015-01-02,2015-01-14,2015-01-17,2015-01-19,2015-01-20,2015-01-23,2015-01-25,2015-01-26,2015-01-28,2015-01-29,2015-01-30,2015-01-31,2015-02-03,2015-02-05,2015-02-10,2015-02-11,2015-02-12,2015-02-17,2015-02-19,2015-02-24,2015-02-25,2015-02-26,2015-03-01,2015-03-03,2015-03-04,2015-03-05,2015-03-10,2015-03-11,2015-03-17,2015-03-18,2015-03-19,2015-03-22,2015-03-24,2015-03-26,2015-03-27,2015-03-31,2015-04-02,2015-04-04,2015-04-08,2015-04-09,2015-04-15,2015-04-16,2015-04-18,2015-04-20,2015-04-21,2015-04-22,2015-04-24,2015-04-26,2015-04-27,2015-04-29,2015-04-30,2015-05-06,2015-05-12,2015-05-19,2015-05-20,2015-05-21,2015-05-25,2015-05-26,2015-05-27,2015-05-28,2015-05-29,2015-06-02,2015-06-03,2015-06-04,2015-06-08,2015-06-09,2015-06-10,2015-06-11,2015-06-16,2015-06-17,2015-06-18,2015-06-22,2015-06-24,2015-06-25,2015-07-02,2015-07-03,2015-07-15,2015-07-18,2015-07-20,2015-07-21,2015-07-23,2015-07-24,2015-07-26,2015-07-27,2015-07-29,2015-07-30,2015-07-31,2015-08-04,2015-08-06,2015-08-11,2015-08-12,2015-08-13,2015-08-18,2015-08-20,2015-08-25,2015-08-26,2015-08-27,2015-09-01,2015-09-02,2015-09-